# Deriving customer insights with SQL and AWS

## Introduction (5 mts)

In this case we'll learn about working large databases that are stored on the cloud and accessed with a database manager. The case will be split into two parts: first, we'll learn the basics of SQL using a smaller data set locally. Then, we'll set up a larger relational database on Amazon Web Services and perform some analysis in the cloud.

# Part 1: Introduction to SQL

**Business Context.** You are a data analyst at a large financial services firm that sells a diverse portfolio of products. In order to make these sales, the firm relies on a call center where sales agents make calls to current as well as prospective customers. The company would like you to dive into their data to devise strategies to increase their revenue or reduce their costs. Specifically, they would like to double down on their most reliable customers.

**Business Problem.** The business would like to answer the following question: **"What types of customers are most likely to buy our product?**

**Analytical Context.** The data is split across 3 tables: "Agents", "Calls", and "Customers", which sit on CSV files. Unlike the last case though, we will first be reading these CSV files into a SQLite database created within Python. You will learn how this database differs from CSV files and how to interact with it using SQL to extract useful insights.

## Background (5 min)

### Why databases?
While we have been dealing with data sitting in CSV files so far, no serious data organization runs their operations off of CSV files on a single person's computer. This practice presents all sorts of hazards, including but not limited to:

1. Destruction of that single device
2. Destruction of the files on that device
3. Inability to connect to that person's device from another device that requires the data
4. Inability to store more than a limited amount of data (since a single device doesn't have that much memory)

Therefore, our data should be stored elsewhere if we want to reliably access it in the future and, more importantly, share it and work on it with others. The **database** is the classic location where modern organizations have chosen to store their data for professional use. A couple of advantages that databases provide are:

- Ability to query only certain recods, instead of fetching and going through the entire csv file
- User based access restrictions - Specify what data each of your users can access from the database. This will strengthen the privacy of the data. 

Daabases have been a topic of research since the late 1960s. Many technology vendors picked up on this and developed databases software for companies to consume. Some of these vendors and products are:

1. Microsoft, initially with Microsoft Access and more recently with Microsoft SQL Server
2. Oracle, with their Oracle database
3. The “PostgreSQL Global Development Group”, with the open-source PostgreSQL

### Types of databases

At this point, you might believe that databases can be thought of as a collection of data. This is true, but unfortunately it is not that simple. Data cannot simply be thrown in a database the same way you throw your socks in your sock drawer. The data that you wish to store in your database must follow some patterns which are determined by the **database type** you wish to store.

#### Relational databases

The most common database type is called a **relational database**, and the systems that manage these kinds of databases are called **Relational Database Management Systems (RDBMS)**. Relational databases date back to the early 1970s and can be considered the first type of database ever conceived.

Relational databases deal with “relational data”, which is a fancy way to say “tabular” data. This kind of dataset consists of rows and columns (i.e. tables) where each row corresponds to an observation and each column corresponds to an attribute of that observation. So, for example, if we go back to the example where we were keeping track of our friends and their phones, each row on the file (or table) represents one friend and each column represents the information we want to track about that friend (name and phone number). The cell on the intersection of the row and column contains the actual data. Relational data is manipulated using a specific language called **SQL (Structured Query Language)**, which we will learn about soon.

A simple way to conceptualize a table inside a relational database is as a CSV file “copied” to the database. In fact, many databases offer that possibility (assuming your file is correctly formatted, of course).

#### NoSQL databases

Around 20 years ago, with the advent of the internet and the necessity to store and process unstructured data (i.e. data that does not fit well in the row-by-column paradigm), developers started to discuss another type of database, which eventually ended up being referred to as a **NoSQL database**. As the name implies, these databases do not rely on SQL and are not relational. They are also built with more “relaxed” rules compared to their predecessors.

## SQL Basics (20 min)

### What is this "SQL" thing?

Just like data can't really survive without a database, a database can't be utilized without SQL. SQL is used for a wide variety of tasks, including but not limited to extracting data, creating the internal structure of a database (in the form of tables), and reading and writing data to these tables.

In this case, we will be writing SQL queries using the `SQLAlchemy` package in Python. This allows you to directly interface with relational databases without exiting the Python environment, while using syntax that is identical to what you would write outside of Python. Run the code below to set up this framework:

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
#maximum number of rows to display
pd.options.display.max_rows = 10

engine=create_engine('sqlite://')
df = pd.read_csv('customers.csv').to_sql('customers', engine, if_exists='replace', index=False)
df = pd.read_csv('agents.csv').to_sql('agents', engine, if_exists='replace', index=False)
df = pd.read_csv('calls.csv').to_sql('calls', engine, if_exists='replace', index=False)

def runQuery(sql):
    result = engine.connect().execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())

We can see what the tables look like:

In [8]:
# display the customers table
queryd1 = """SELECT *
FROM customers
"""
runQuery(queryd1)

,customerid,name,occupation,email,company,phonenumber,Age
0,0,David Melton,Unemployed,DMelton@zoho.com,"Morris, Winters and Ramirez",409-093-0748,16
1,1,Michael Gonzalez,Student,Gonzalez_Michael@yahoo.com,Hernandez and Sons,231-845-0673,19
2,2,Amanda Wilson,Student,Amanda.Wilson75@verizon.com,"Mooney, West and Hansen",844-276-4552,18
3,3,Robert Thomas,"Engineer, structural",RThomas@xfinity.com,Johnson-Gordon,410-404-8000,25
4,4,Eddie Hall,Surgeon,EddieHall@outlook.com,Dawson LLC,872-287-2196,30
...,...,...,...,...,...,...,...
995,995,Ashley Young,Student,Ashley_Y@xfinity.com,Esparza-Johnson,751-654-6719,19
996,996,Mr. Steven Smith,"Engineer, structural",Mr..Smith@zoho.com,Hensley-Odom,279-898-4565,26
997,997,Mark Smith,"Engineer, control and instrumentation",Mark_S@yahoo.com,Fisher LLC,563-382-1868,29
998,998,Jeffrey Carrillo,Unemployed,JeffreyCarrillo@yahoo.com,Morgan LLC,223-784-2416,15


In [11]:
# display the calls table
queryd2 = """SELECT *
FROM calls
"""
runQuery(queryd2)

,callid,agentid,customerid,pickedup,duration,productsold
0,0,10,179,0,0,0
1,1,5,691,1,116,0
2,2,10,80,1,165,0
3,3,6,629,1,128,0
4,4,8,318,1,205,0
...,...,...,...,...,...,...
9935,9995,6,92,1,103,0
9936,9996,0,731,1,188,0
9937,9997,4,53,1,152,0
9938,9998,5,260,0,0,0


### Exploring the `customers` table

The most important thing you will ever do in SQL is extract a subset of the data from a SQL table based on a set of rules. This is accomplished using the following statement syntax:

1. Start with the keyword `SELECT`
2. Follow with the names of the columns you want to select, separated by commas (alternatively, you can use the `*` symbol to indicate you wish to select all columns)
3. Follow with the keyword `FROM`
4. Finish with the name of the table you wish to select data from
	
Additionally, you can use the `WHERE` clause to only return results which satisfy certain conditions (similar to how code within Python if-then blocks only execute if the associated conditions are true). `WHERE` clauses immediately follow the table name you want to select data from.

Since the firm wants to dig deeper into its customers, let's start by pulling some of their data out of our files; namely, information about customers who are not unemployed (and therefore are more likely to buy from us).

### Exercise 1: (5 min)

Write a query that selects the customer ID and name from the `customer` table, only showing results for customers who are not unemployed. Remember to write your query as a multi-line string (enclosed within a pair of triple quotes `"""`) and pass it to the `runQuery()` function defined in the framework above to check your work!

**Answer**. One possible solution is given below:

```SQL
SELECT customerid, name
FROM customers
WHERE occupation != 'Unemployed'
```

In [4]:
query1 = """SELECT customerid, name
FROM customers
WHERE occupation != 'Unemployed'"""
runQuery(query1)

,customerid,name
0,1,Michael Gonzalez
1,2,Amanda Wilson
2,3,Robert Thomas
3,4,Eddie Hall
4,6,Maria Johnson
...,...,...
755,994,Ruben Steele
756,995,Ashley Young
757,996,Mr. Steven Smith
758,997,Mark Smith


Of course, for names, it's sensible to try to list them in alphabetical order. SQL allows us to do this rather easily with the `ORDER BY` statement. This is then followed by a comma-separated list of columns on which you want to order your results (columns that come first take priority in the subsequent ordering). Optionally, you can then append the keyword `ASC` or `DESC` (short for ascending and descending, respectively) after each column to determine the ordering type (e.g. alphabetical or reverse-alphabetical for a string column).

We can also use the `AS` statment to change the name of a column returned by your query. However, this change is only temporary and is only valid for that particular query. For example, we can rename the `name` column to `customername` and order it alphabetically. This operation is known as **aliasing**:

```SQL
SELECT customerid, name AS customername
FROM customers
WHERE occupation != 'Unemployed'
ORDER BY customername
```

In [5]:
query2 = """SELECT customerid, name AS customername
FROM customers
WHERE occupation != 'Unemployed'
ORDER BY customername"""
runQuery(query2)

,customerid,customername
0,900,Aaron Gutierrez
1,622,Aaron Rose
2,226,Adam Ward
3,786,Alan Chambers
4,985,Alan Mitchell
...,...,...
755,699,Willie Greene
756,715,Yesenia Wright
757,952,Yolanda White
758,421,Zachary Ruiz


This is a great first step; however, while producing the list of customers that are not unemployed, you inevitably spend a lot of time looking at the different professions your customers have and realize how often engineers appear in your database. You know that engineering jobs tend to command higher salaries these days, so you decide to try to extract a list of all the unique types of engineering jobs that are represented in your database. To ensure that you don't get duplicate job titles in your query results, you'll need to write the keyword `DISTINCT` immediately after `SELECT` in your query.

### Exercise 2: (5 min)

Write a query which produces a list, in alphabetical order, of all the distinct occupations in the `customer` table that contain the word "Engineer".

(Hint: The `LIKE` operator can be used when you want to look for similar values. It is included as part of a `WHERE` clause. It needs to be complemented with the `%` symbol, which is a wild card that represents zero, one, or multiple characters. For example, one valid `WHERE` clause utilizing the `LIKE` operator is `WHERE name LIKE 'Matt%'`, which would return any results where the person's name starts with the word "Matt"; e.g. "Matt" or "Matteo" or "Matthew", etc.)

**Answer.** One possible solution is given below:

```SQL
SELECT DISTINCT occupation
FROM customers
WHERE occupation LIKE '%Engineer%'
ORDER BY occupation
```

In [6]:
query3 = """SELECT DISTINCT occupation
FROM customers
WHERE occupation LIKE '%Engineer%'
ORDER BY occupation"""
runQuery(query3)

,occupation
0,Chemical engineer
1,Electrical engineer
2,"Engineer, aeronautical"
3,"Engineer, agricultural"
4,"Engineer, automotive"
...,...
24,"Engineer, production"
25,"Engineer, site"
26,"Engineer, structural"
27,"Engineer, technical sales"


Now, one of your marketing colleagues tells you that people who are 30 or older will have a higher probability of buying your product (presumably because by that point they have more disposable income and savings). You don't want to take your colleague's word for granted, so you decide not to completely ignore people under 30, but instead to add that information on the report regarding the person’s age, so that the agent making the subsequent call can decide how they want to use that information. However, due to privacy concerns, you also cannot share the person's exact age.

### Exercise 3: (5 min)

Write a query that retuns the customer ID, their name, and a column `Over30` containing "Yes" if the customer is more than 30 years of age and "No" if not.

(Hint: You will need to use the `CASE-END` clause. The `CASE-END` clause can be used to evaluate conditional statements and returns a value once a condition is met (similar to an if-then-else clause in Python). If no conditions are true, it returns the value in the ELSE clause (or NULL if there is no ELSE statement). For example:

```SQL
CASE
    WHEN name = "Matt" THEN 'Yes'
    WHEN name = "Matteo" THEN 'Maybe'
    ELSE 'No'
END
```

**Answer.** One possible solution is given below:

```SQL
SELECT customerid, name,
    CASE
        WHEN age >= 30 THEN 'Yes'
        WHEN age <  30 THEN 'No'
        ELSE 'Missing Data'
    END AS Over30
FROM customers
ORDER BY name DESC
```

In [7]:
query4 = """SELECT customerid, name,
    CASE
        WHEN age >= 30 THEN 'Yes'
        WHEN age <  30 THEN 'No'
        ELSE 'Missing Data'
    END AS Over30
FROM customers
ORDER BY name DESC"""
runQuery(query4)

,customerid,name,Over30
0,392,Zachary Wilson,Yes
1,986,Zachary Stevenson,No
2,421,Zachary Ruiz,Yes
3,18,Zachary Howe,No
4,883,Zachary Anderson,No
...,...,...,...
995,65,Adam Jimenez,No
996,622,Aaron Rose,No
997,145,Aaron Mcintyre,No
998,461,Aaron Hendrix,No


Let's now modify Exercise 3 so that the query only returns customers who work in an engineering profession:

```SQL
SELECT customerid, name,
    CASE
        WHEN age >= 30 THEN 'Yes'
        WHEN age <  30 THEN 'No'
        ELSE 'Missing Data'
    END AS Over30
FROM customers
WHERE occupation LIKE '%Engineer%'
ORDER BY name DESC
```

In [8]:
query5 = """SELECT customerid, name,
    CASE
        WHEN age >= 30 THEN 'Yes'
        WHEN age <  30 THEN 'No'
        ELSE 'Missing Data'
    END AS Over30
FROM customers
WHERE occupation LIKE '%Engineer%'
ORDER BY name DESC"""
runQuery(query5)

,customerid,name,Over30
0,421,Zachary Ruiz,Yes
1,952,Yolanda White,No
2,699,Willie Greene,Yes
3,973,William Jackson,Yes
4,966,William Garcia,No
...,...,...,...
356,918,Alison Vaughan,Yes
357,568,Alice Lee,No
358,432,Alexis Riddle,No
359,985,Alan Mitchell,Yes


## Investigating customer conversion rates (30 min)

In order to validate whether our hypotheses about engineers and age are true (for example, engineers exhibit higher product sales conversion rates, and perhaps engineers over 30 tend to exhibit an even higher conversion rate), we will need to use two tables: `calls` and `customers`. This is because the column `productsold` lies only in the `calls` table, yet information about customer professions and age only lie in the `customers` table.

`SELECT` commands are not restricted to a single table. In fact, theoretically, there is no limit to the number of tables that you can extract data from in a single SQL query. Let's introduce some new concepts that are relevant once we go beyond a single table.

**Primary and foreign keys** are very important concepts that need to be understood by any database professional. Primary keys:

1. Uniquely identify a record in the table. Their name usually includes the word "id"
    * For example, `customerid` is the primary key of the `customers` table, `agentid` is the primary key of the `agents` table, and `callid` is the primary key of the `calls` table    
2. Do not accept null values. And they shouldn't, because they are being used to identify the record
3. Are limited to one per table

On the other hand, foreign keys:

1. Are a field in the table that is the primary key in another table
2. Can accept null values
3. Are not limited in any way per table
    * For example, the `calls` tables has 2 foreign keys: `agentid` and `customerid` pointing to the `agents` and `customers` tables, respectively

### Extracting call data for customers working in engineering professions (10 min)

Let's first extract the relevant data so we can perform this analysis. Here, a `JOIN` clause will come in handy. A `JOIN` clause consists of two parts:

1. The base `JOIN` statement, which is of the form `[Table 1] JOIN [Table 2]`. This performs a Cartesian product on the 2 tables being joined. For example, if we have Table A with 5 rows, and Table 5 with 3 rows, their Cartesian product will return 15 rows (5 x 3)
2. A `JOIN` criteria, which filters the Cartesian product's results, beginning with the `ON` keyword

Here is an example of a `JOIN` criteria in action, which is telling us to only give combinations of rows where the agent ID matches in both tables:

```SQL
SELECT callid, a.agentid, name
FROM calls c
JOIN agents a ON c.agentid = a.agentid
ORDER BY name DESC
```

In [9]:
query6 = """SELECT callid, a.agentid, name
FROM calls c
JOIN agents a ON c.agentid = a.agentid
ORDER BY name DESC"""
runQuery(query6)

,callid,agentid,name
0,12,3,Todd Morrow
1,28,3,Todd Morrow
2,32,3,Todd Morrow
3,50,3,Todd Morrow
4,60,3,Todd Morrow
...,...,...,...
9934,9985,10,Agent X
9935,9986,10,Agent X
9936,9991,10,Agent X
9937,9992,10,Agent X


Note that:

1. `c` and `a` are aliases to the `calls` and `agents` tables to avoid having to type the table name every time. Unlike with column aliasing earlier, we do not need the `AS` keyword here
2. We write `a.agentid` instead of `agentid` in the SELECT statement – this is because the `agentid` column exists in both tables, so we have to tell the database which one to get the result from

### Exercise 4: (5 min)

Write a query which returns all calls made out to customers in the engineering profession, and shows whether they are over or under 30 as well as whether they ended up purchasing the product from that call.

**Answer.** One possible solution is given below:

```SQL
SELECT callid, cu.customerid, name, productsold,
    CASE
        WHEN age >= 30 THEN 'Yes'
        WHEN age <  30 THEN 'No'
        ELSE 'Missing Data'
    END AS Over30
FROM customers cu
JOIN calls ca ON ca.customerid = cu.customerid
WHERE occupation LIKE '%Engineer%'
ORDER BY name DESC
```

In [10]:
query7 = """SELECT callid, cu.customerid, name, productsold,
    CASE
        WHEN age >= 30 THEN 'Yes'
        WHEN age <  30 THEN 'No'
        ELSE 'Missing Data'
    END AS Over30
FROM customers cu
JOIN calls ca ON ca.customerid = cu.customerid
WHERE occupation LIKE '%Engineer%'
ORDER BY name DESC"""
runQuery(query7)

,callid,customerid,name,productsold,Over30
0,2049,421,Zachary Ruiz,0,Yes
1,2960,421,Zachary Ruiz,0,Yes
2,3365,421,Zachary Ruiz,0,Yes
3,3386,421,Zachary Ruiz,1,Yes
4,4332,421,Zachary Ruiz,0,Yes
...,...,...,...,...,...
3614,6444,622,Aaron Rose,1,No
3615,7994,622,Aaron Rose,0,No
3616,8811,622,Aaron Rose,0,No
3617,9524,622,Aaron Rose,1,No


### Analyzing the call conversion data (20 min)

Now, we want to determine whether or not customers in our desired cohort exhibit a higher sales conversion rate compared to the overall population of customers. A reasonable way to do this is to count the total number of calls to this cohort which resulted in a sale, and divide that by the total number of calls to this cohort (whether or not they resulted in a sale) to get a percentage, and then compare that with the percentage we compute from the `calls` table overall.

However, to compute these figures, we'll need to learn a bit about **aggregation functions**. An aggregation function allows you to perform a calculation on a set of values to return a single value, essentially computing some sort of summary statistic.

The following are the most commonly used SQL aggregate functions:

1. `AVG()` – calculates the average of a set of values
2. `COUNT()` – counts rows in a specified table or view
3. `MIN()` – gets the minimum value in a set of values
4. `MAX()` – gets the maximum value in a set of values
5. `SUM()` – calculates the sum of values

### Exercise 5: (10 min)

Write two queries: one that computes the total sales and total calls made to customers in the engineering profession, and one that computes the same metrics for the entire customer base. What can you conclude regarding the conversion rate within the engineering customers vs. the overall customer base?

**Answer.** One possible solution is given below:

```SQL
SELECT SUM(productsold) AS totalsales, COUNT(*) AS ncalls
FROM customers cu
JOIN calls ca ON ca.customerid = cu.customerid
WHERE occupation LIKE '%Engineer%'
```

In [11]:
query8 = """SELECT SUM(productsold) AS totalsales, COUNT(*) AS ncalls
FROM customers cu
JOIN calls ca ON ca.customerid = cu.customerid
WHERE occupation LIKE '%Engineer%'"""
runQuery(query8)

,totalsales,ncalls
0,760,3619


```SQL
SELECT SUM(productsold) AS totalsales, COUNT(*) AS ncalls
FROM customers cu
JOIN calls ca ON ca.customerid = cu.customerid
```

In [12]:
query9 = """SELECT SUM(productsold) AS totalsales, COUNT(*) AS ncalls
FROM customers cu
JOIN calls ca ON ca.customerid = cu.customerid"""
runQuery(query9)

,totalsales,ncalls
0,2084,9925


The conversion rate for both groups is ~20.9%, indicating that engineers are not more likely to purchase our products than the overall population.

### Exercise 6: (5 min)

Write a query that computes the total sales and total calls made to customers over the age of 30. Is there a notable difference between the conversion ratio here and that of the overall customer base?

**Answer.** One possible solution is given below:

```SQL
SELECT SUM(productsold) AS totalsales, COUNT(*) AS ncalls
FROM customers cu
JOIN calls ca ON ca.customerid = cu.customerid
WHERE age >= 30
```

In [13]:
query10 = """SELECT SUM(productsold) AS totalsales, COUNT(*) AS ncalls
FROM customers cu
JOIN calls ca ON ca.customerid = cu.customerid
WHERE age >= 30"""
runQuery(query10)

,totalsales,ncalls
0,659,3096


The conversion rate is ~21.1% vs. the overall ~20.9%. There may be some difference, but it is quite small so we would need to run statistical significance tests in order to validate this. Since that's not the focus of this case, we'll skip that for now.

### Exercise 7: (5 min)

How about if you look at the sales conversion rate for engineers over the age of 30?

**Answer.** One possible solution is given below:

```SQL
SELECT SUM(productsold) AS totalsales, COUNT(*) AS ncalls
FROM customers cu
JOIN calls ca ON ca.customerid = cu.customerid
WHERE occupation LIKE '%Engineer%' AND age >= 30
```

In [14]:
query11 = """SELECT SUM(productsold) AS totalsales, COUNT(*) AS ncalls
FROM customers cu
JOIN calls ca ON ca.customerid = cu.customerid
WHERE occupation LIKE '%Engineer%' AND age >= 30"""
runQuery(query11)

,totalsales,ncalls
0,376,1816


Here, we actually observe the opposite pattern – the conversion rate is only ~20.5%.

From these numbers, we can conclude that a customer's status as an engineering professional has no positive effect on their conversion rate. On the other hand, having an age of at least 30 MAY have some effect; however, we would need to do more in-depth statistical testing to determine this.

# Transition (10 min)

## What is AWS?

Amazon Web Services (AWS) is a cloud service from Amazon, which provides services in the form of building blocks. These building blocks can be used to create and deploy any type of application in the cloud.

These services are designed to work with each other, and result in applications which are sophisticated and highly scalable. The following are the most commonly used domains:

- The **Compute** domain includes services related to compute workloads. Services in this domain can be used to run computationally intensive or repetitive tasks that you don't want to run locally
- The **Database** domain is used for database related workloads. Services in this domain provide cost-efficient and resizable capacity and can automate time-consuming administration tasks such as provisioning hardware, setting up the database, patching, and making backups
- The **Migration** domain is used for transferring data to or from the AWS Infrastructure
- The **Networking and Content Delivery** domain is used for isolating your network infrastructure, and content delivery is used for faster delivery of content
- The **Management Tools** domain consists of services which are used to manage other services in AWS
- The **Security & Identity, Compliance** domain consist of services which are used to manage to authenticate and provide security to your AWS resources
- The **Messaging** domain consists of services which are used for queuing, notifying or emailing messages

In this case we'll focus on the database domain, specifically the RDS service. You can find more information about other AWS services in the Additional Resources section at the end of the case.

## Log In! 

Start by signing in to [AWS](https://signin.aws.amazon.com/signin?redirect_uri=https%3A%2F%2Fconsole.aws.amazon.com%2Fconsole%2Fhome%3Fnc2%3Dh_ct%26src%3Dheader-signin%26state%3DhashArgs%2523%26isauthcode%3Dtrue&client_id=arn%3Aaws%3Aiam%3A%3A015428540659%3Auser%2Fhomepage&forceMobileApp=0) using your root user credentials. If you have not created your own AWS account, please consult with your TA to obtain the relevant instructions.

# Part Two: Analyzing Net Promoter Score (NPS) data with AWS

## Introduction (5 min)

**Business Context.** You are a data scientist at a new but fast-growing startup. The startup released its first product 12 months ago and has been tracking Net Promoter Score (NPS) over its growing customer base since the product's launch.

The team assumes that the NPS score is correlated to the product stability and feature-completeness and that the product has been getting more stable and complete over time. They also realize that there have been some hiccups along the way, and they assume that NPS has therefore fluctuated up and down.

**Business Problem.** The startup wants you to investigate the data and answering the following question: **"Has our NPS improved over time? And has our average NPS decreased in specific periods over the last 12 months?"**

**Analytical Context.** In this part of the case, you will be working with a large dataset – so large that your personal laptop is not powerful enough to run heavy SQL queries on it (the startup is stingy and doesn't provide employees with hardware – luckily they have free cloud credits though!). Instead, you will be working with a powerful PostgreSQL database in the cloud (on Amazon Web Services), and uploading the data there for remote processing. We'll connect to the remote database and have the remote machine run the resource-intensive queries.

## Understanding the Net Promoter Score (NPS) (10 min)

NPS is a metric to measure customer satisfaction. You've probably seen pop-ups online, or received surveys via email, asking you "Would you recommend [product] to a friend or family member?" and giving you the option to respond with a number between 0 and 10. That's someone collecting information to calculate their NPS.

![nps Example Survey](images/nps-example-survey.png)

The basic idea is simple - customers who respond with high ratings are more likely to promote your product to other potential customers. Customers who give low ratings are unhappy and are unlikely to help you grow your customer base. If you ask enough people at different time periods, you can track customer satisfaction over time and see how this correlates to product development and other aspects of your business that are within your control. 

NPS categorizes users into three groups based on the ratings that they leave. This is done as follows:

1. Users who leave a rating of 0 - 6 are regarded as "detractors"
2. Users who leave a rating of 7 or 8 are regarded as "passives"
3. Users who leave a rating of 9 or 10 are regarded as "promoters"

The final NPS score for a given period is calculated as the percentage of total users who are promoters minus the percentage of total users who are detractors. This means that an NPS score can be anything from -100 to 100.

### Exercise 8: (5 min)

If you have the following scores left by your customers:

| Date           | CustomerId | Score | Group     |
| -------------- | ---------- | ----- | --------- |
| 1 January 2018 | 562        | 1     | Detractor |
| 1 January 2018 | 544        | 10    | Promoter  |
| 2 January 2018 | 333        | 9     | Promoter  |
| 2 January 2018 | 102        | 9     | Promoter  |
| 4 January 2018 | 267        | 9     | Promoter  |
| 5 January 2018 | 981        | 10    | Promoter  |
| 6 January 2018 | 105        | 6     | Detractor |
| 6 January 2018 | 459        | 7     | Passive   |
| 6 January 2018 | 188        | 10    | Promoter  |
| 8 January 2018 | 982        | 8     | Passive   |

What is your NPS? How would you adjust your calculation of NPS if instead users had many opportunities to rate you in a short time period? What would you consider to be a "good" NPS? 

**Answer.** We have 10 responses: 6 promoters, 2 detractors and 2 passives. That is 60% promoters and 20% detractors, so our NPS is $60 - 20 = 40$.

If users could rate us many times in a short time span, a sensible adjustment would be to first average all of the responses per user, and use this averaged response to group each user into promoter, detractor, or passive. This is because no matter how many times a single user interacts with our product, they are likely still only paying us once for it. Thus, since they are not weighted more heavily in our revenue streams, they should not be weighted more heavily in our customer satisfaction schemes either.

Defining what constitutes a good NPS depends on the specific line of business the company is in. It varies between [different industries](https://www.qualtrics.com/marketplace/nps-benchmarks/) with internet providers generally getting far lower scores than technology companies.

## Setting up a cloud database using RDS and importing data (30 min)

Ok! Let's set up a database and load in some NPS data so that we can analyze it using SQL. We'll use the code at [this repository](https://github.com/sixhobbits/nps-sample-data) to generate a large sample of fake NPS data and push it into a PostgreSQL instance running in the cloud. (Don't look at the source code that generates the data, as it will spoil the fun.)

1. Log into your AWS account and select "RDS" from the service list. You should see a screen like the one below, where you can hit the "Create database" button:

![Create Database](images/create_db.png)

2. The next option you'll see asks you if you want to use "standard create" or "easy create". Easy might sound tempting, but **choose "standard"** as we'll have to set up our database for public use so we can connect to it locally.

3. Choose "PostgreSQL" as the database type, leave the version at the default AWS has chosen for you (10.6-R1 at the time of writing), and choose "Free Tier"

4. Under the next section, choose a name for your database instance. Remember this is the machine that is hosting the database software, not the database itself (one RDS instance can host many databases), so I'm calling mine `nps-demo-instance` to reflect this, although we'll only be creating a single database for now. 

5. You can leave the master username as `postgres` and ask RDS to autogenerate a password (we'll be able to see this password at the next step):

![Set DB password](images/set_db_password.png)

6. You can leave the next settings as their defaults until you get to the "Connectivity" section. Usually, you'll set up an RDS instance to play with other infrastructure within your AWS account, such as EC2 servers. In our case, we want to push data in and out of the database directly from our local machine as the client, so we'll have to set our database up for "public access". This is generally less secure, but we'll add some firewall rules in a bit to make sure that only we can access it:

      * Expand the "Additional connectivity configuration" section

      * Set "publicly accessible" to "Yes"

      * Under "VPC security group", choose to "Create new", and give it a name like `allow-local-access`. This will create a firewall rule that will allow you to connect to your database on port 5432 (the default for PostgreSQL) using your current IP address. If you are using public WiFi, a hotspot, or if you think your IP address is likely to change soon for any reason, note that you'll have to modify this security group any time your IP address changes:

![Create Security Group](images/create-sec-group.png)

7. Press the "Create database" button in the bottom right, and you'll be taken back to the overview page where you can see your database being created. At the top, there'll be a notification where you can press "View credential details" to access your master password that was automatically generated. Take note of this as you can only see it once. NOTE: this creates a database in the default VPC. If your default VPC is not configured for DNS connections, you will need to create a new VPC. Please see 'Appendix 1: Troubleshooting RDS creation' for instructions on how to do achieve this.

![View credentials](images/view_creds.png)

8. Once your database becomes "available" (you might need to press the "refresh" button indicated below to see the change), you can connect to it. Click on the name of the database (`nps-demo-instance` in our example), to find out the connection details:

![DB available](images/db-available.png)

9. Once you click on the database, you should see the endpoint that you need on a screen similar to the one shown below. You need this endpoint to connect to the database from your local machine.

![DB Endpoint](images/db-endpoint.png)

10. Locally, open a terminal and run the following command, substituting [endpoint] with the one that you noted from the RDS console above.

```bash
psql -h [endpoint] -U postgres
```

This will connect to our instance's default database using the master username. It will prompt you for the password and you can enter the autogenerated password from above. You should now see a SQL prompt, similar to the image below:

![PSQL prompt](images/psql-prompt.png)

We've successfully created a cloud database and connected to it!

### Setting up our NPS database (10 min)

Let's proceed by setting up our database in Amazon RDS:

1. In the SQL shell, run the following commands to create a database, create a user to manage our database, and give privileges on our new database to our new user. Replace [password] with your own choice of password:

```SQL
create database nps_demo_db;
create user nps_demo_user with login encrypted password '[password]';
grant all privileges on database nps_demo_db to nps_demo_user;
\q
```

Here, `\q` closes the connection so you can re-open it under a different user.

2. Run the following command. It is similar to the one we used before to connect but now specifies both our custom user and our custom database. Once again, substitute [endpoint] with the one you see in the RDS console.

```SQL
psql -h [endpoint] -U nps_demo_user -d nps_demo_db
```

3. Put in the new password that you entered in the SQL statement in step 1 instead of the master password that AWS automatically generated for us.  You'll see a very similar prompt, but with the `nps_demo_db=>` prompt instead of `postgres=>`:

![nps demo prompt](images/nps-demo-prompt.png)

The next thing we need to do is to create tables to house our data. We'll use the data from [this repository](https://github.com/sixhobbits/nps-sample-data/), consisting of two tables: `customer` and `score`. There are some extra fields on `customer` (`is_premier` and `is_spam`) that we won't use right away, but we'll create our tables to match that format anyway to make the import easier.

The important context is that we are imagining a scenario where:

* We have been running a new company for around one year.
* The product has gone through different stages of feature improvement and stability but has overall shown growth and improvement.
* Every day, new customers join and both new and old customers may or may not leave us a score between 0-10 to rate how likely they are to recommend our product to family and friends.
* At the start and at some key points during the year, the product is unstable or lacking features and this affects the customer rating.

Now, lets use SQL to create two tables: one for customers, and one for the scores that our customers leave.

To create the `customer` table, we do the following:

```SQL
create table customer (id serial not null, created_at date, is_premier boolean, is_spam boolean, CONSTRAINT customer_pkey PRIMARY KEY (id));
```

This creates a `customer` table with an ID, the date the customer first signed up (`created_at`), and two boolean flags that we don't need yet. It also adds a constraint to the ID field saying it is a primary key, meaning it has to be unique. 


To create the table of scores that our customers leave, use the following command in the same prompt:

```SQL
create table score (id serial not null, customer_id integer references customer(id), created_at date, score integer, CONSTRAINT scores_pkey PRIMARY KEY (id));
```

This is similar to the `customer` table, but has a `score` field to store the value between 0 and 10 that a customer leaves each time they complete a survey, and another date field to record when the survey was done. There is also a foreign key `customer_id` to link each score to a specific entity in the `customer` table. 

You can close the connection again with `\q`.

### Pushing sample data into RDS (10 min)

Let's now push the NPS data onto RDS:

1. Download the two CSV files (`score.csv` and `customer.csv`) from https://github.com/sixhobbits/nps-sample-data into your local working directory. Don't look at the README file.

2. Run the command below, again substituting [endpoint] with the actual endpoint you used above. Make sure that the `customer.csv` file is located in the same directory that you run the `psql` command from:

```bash
psql -h [endpoint] -U nps_demo_user -d nps_demo_db -c "\copy customer from 'customer.csv' with (format csv, header true, delimiter ',');"
```

The first part of the command is the same one we used before to open a SQL shell. Here we also pass the `-c` flag which allows us to specify a SQL command to be run on the database. Because our shell has permissions to access our local file system, but our database doesn't, running the command like this means we won't have problems with permissions. In the `\copy` command, we specify which table we want to populate (`customer`), where the local file is (`customer.csv`), that our file is in CSV format, that it has a header, and that we are using a comma as a delimiter. 

This should prompt you for the password (again, use the one that you created for the `nps_demo_user`). It will then let you know how many rows it has successfully imported, similar to the image below:

![Copy successful](images/copy-successful.png)

3. Now we can add the scores data as well using the same method. The only things we need to change are the table name and the filename from which we source the data. The full command (don't forget to substitute your endpoint) is:

```bash
psql -h [endpoint] -U nps_demo_user -d nps_demo_db -c "\copy score from 'score.csv' with (format csv, header true, delimiter ',');"
```

There are a lot more sample scores than customers (as each customer can respond to the survey more than once), so this will take a bit longer than the previous command:

![Import scores](images/import-scores.png)

## Analyzing our NPS data using SQL (40 min)

Now we can proceed to the fun part. We have NPS scores left by a large number of customers over the past year, and we want to see how these scores change over time.

We only have raw data – numbers between 0 and 10 inclusive – so we'll use SQL to group this data in different ways and transform it into NPS data. If you remember how to define NPS from the first section, you can probably work out that the main things we need to do are:

1. Break down our scores per customer for any given time period (here, we will look at this per week)
2. Divide customers into promoters, passives or detractors, based on the scores they have left in that week
3. Calculate the NPS per week and look at how this value changes week-by-week

### Counting customers and scores (10 min)

We saw how many customers and scores we had when we did the import step above. However, in a real-world setting, you would have gathered this data slowly, over time, so let's start by counting out customers, our survey responses (`scores`), and looking at how many surveys each customer responds to. For each of the following, you'll need to be connected to the SQL shell, so run the following first (using your endpoint) and any time you need to.

We'll show the output of each SQL command directly below – you only need to enter the command shown in the first section in each of the following examples.

```bash
psql -h [endpoint] -U nps_demo_user -d nps_demo_db
```

#### Counting customers

```SQL
SELECT COUNT(*) FROM customer;
```

```
 count
--------
 188323
(1 row)
```

We have nearly 200k customers, which is not bad for a product that's been running for one year!

#### Counting scores

```SQL
SELECT COUNT(*) FROM score;
```

```
  count
---------
 1577578
(1 row)
```

And we have over 1.5 million survey responses. That's just over 8 responses per customer if we assume an equal distribution. Let's use SQL to look at that.

### Exercise 9: (5 min)

Write a SQL query that outputs a table showing the 10 customers with the highest number of responses and their total response count, in descending order (customer with most responses at the top).

**Answer.** One possible solution is given below:

```SQL
SELECT customer_id, COUNT(score.id) AS cnt FROM score
INNER JOIN customer ON customer_id = customer.id
GROUP BY customer_id ORDER BY cnt DESC
LIMIT 10;
```

```
 customer_id | count
-------------+-------
          31 |    38
         928 |    38
        4271 |    38
        5333 |    37
        1253 |    37
        1259 |    36
        1030 |    36
        2327 |    36
         564 |    36
        2335 |    36
```

We can also use SQL JOINs using commas and a `WHERE` clause as a shortcut. The above command is equivalent to the following one, but the earlier version is preferable in most contexts as it is more explicit:

```SQL
SELECT customer_id, COUNT(score.id) AS cnt FROM score, customer
WHERE customer_id = customer.id
GROUP BY customer_id ORDER BY cnt DESC
LIMIT 10;
```



We can also look at customers who have left very few responses by ordering by `ASC` instead of `DESC`:

```SQL
SELECT customer_id, COUNT(score.id) AS cnt FROM score
INNER JOIN customer ON customer_id = customer.id
GROUP BY customer_id ORDER BY cnt ASC
LIMIT 10;
```

```
 customer_id | cnt
-------------+-----
       57565 |   1
       62357 |   1
       49021 |   1
       57424 |   1
       61891 |   1
       62295 |   1
       44796 |   1
       44995 |   1
       57286 |   1
       62402 |   1
```

We can see there are at least 10 customers who have left only a single response. Let's do a 'count of counts' query to get a better idea of how many responses most customers leave. We want to count how many customers have left exactly $x$ responses. 

### Exercise 10: (5 min)

Write a SQL query that outputs a table showing how many customers leave $x$ responses for any given integer $x$. Sort this table in descending order ($x$ with highest number of customers leaving $x$ responses at the top).

(Hint: Use a **nested** `SELECT` statement. A nested statement is when you treat the results of one query as the input to another one.)

**Answer.** One possible solution is given below:

```SQL
SELECT cnt, COUNT(cnt) as count_of_count FROM
(SELECT customer_id, count(score.id) AS cnt FROM score
INNER JOIN customer ON customer_id = customer.id
GROUP BY customer_id ) a
GROUP BY cnt
ORDER BY count_of_count DESC
LIMIT 100;
```

Notice in the query above we have taken a query very similar to the one from Exercise 10 and nested it in parentheses. We have then given this intermediate query an **alias**, which comes immediately after the closing parenthesis; in this case we have chosen the alias `a`. It is a common convention to use aliases `a`, `b`, `c`, etc. as a shorthand if you are primarily interested only in the final result.

From our previous queries, we already know that all the values have to fall between 1 and 38, so there can be a maximum of 38 rows returned in this query. Therefore there is no real need to add a LIMIT clause, but we add a `LIMIT 100` anyway. This is a good habit in case you make a wrong assumption to prevent the case where you accidentally try to pull thousands or millions of rows from a remote server. For brevity, we only included the first 15 rows of output below:

```
 cnt | count_of_count
-----+----------------
   6 |          18779
   5 |          17218
   7 |          17094
   4 |          15642
   8 |          14108
   3 |          12983
   9 |          11978
  10 |          10556
   2 |          10191
  11 |           9001
  12 |           7833
  13 |           6698
   1 |           6302
  14 |           5707
  15 |           4908
```

We can see that most customers leave between 2 and 10 responses so the maximum of 38 is an outlier. A fair number of people only leave one response.

### Average scores per week (10 min)

However, we still have not looked at how scores are *changing*. Let's average all scores in each week and see how the scores go up and down over time:

```SQL
SELECT TO_CHAR(score.created_at, 'IYYY-IW') AS week, AVG(score) AS avg_score
FROM score
GROUP BY week
ORDER BY week ASC
LIMIT 100;
```

Again, we did not need to add a limit clause as we know there will only be 52 rows (the number of weeks in a year, which is the span of our dataset), but we do anyway for good measure and again include only the first 15 rows of output below:

```
  week   |     avg_score
---------+--------------------
 2018-01 | 5.3618090452261307
 2018-02 | 6.1577181208053691
 2018-03 | 5.1405228758169935
 2018-04 | 5.2256097560975610
 2018-05 | 6.3962765957446809
 2018-06 | 7.2065359477124183
 2018-07 | 7.0110294117647059
 2018-08 | 6.9827490261547023
 2018-09 | 7.4689516129032258
 2018-10 | 7.9564362001124227
 2018-11 | 8.0201993704092340
 2018-12 | 7.8336310283235519
 2018-13 | 7.9298795180722892
 2018-14 | 7.9583184257602862
 2018-15 | 7.9876211782252051
```

We can see that the scores start low and generally trend up over time, although they go down again around week 36 (not shown above). We use the [ISO Week](https://en.wikipedia.org/wiki/ISO_week_date) through PostgreSQL's `TO_CHAR` function to break down each of our dates into a specific week number and average the scores per week. 

There are a couple issues with the above query, though:

1. The `AVG` function shows a lot of decimal points by default which makes it more difficult to read the data
2. Many customers leave a different number of responses and some might leave more than one response per week

A good compromise is to calculate the average score per customer per week, then average all of these to get an average score across all customers per week. Let's do this and round off some decimal points to make our data easier to read.

### Exercise 11: (5 min)

Write a query to compute the average score across all customers per week, rounding off to two decimal places. (Hint: Use the `ROUND()` function, which takes two arguments: the quantity you are rounding, and how many decimals you are rounding off to.)

**Answer.** One possible solution is given below:

```SQL
SELECT week, ROUND(AVG(avg_week_score),2) as avg_score FROM
(SELECT TO_CHAR(score.created_at, 'IYYY-IW') AS week, customer_id, AVG(score) as avg_week_score FROM score
GROUP BY week, customer_id) a
GROUP BY week
ORDER BY week
LIMIT 100;
```

```
 week   | avg_score
---------+-----------
 2018-01 |      5.12
 2018-02 |      5.80
 2018-03 |      5.74
 2018-04 |      5.50
 2018-05 |      6.33
 2018-06 |      7.02
 2018-07 |      7.01
 2018-08 |      6.89
 2018-09 |      7.38
 2018-10 |      7.75
 2018-11 |      7.80
``` 

### Classifying our customers as promoters, passives, or detractors (10 min)

Now, let's proceed to classifying our customers so we can calculate the NPS per week. We used a similar `SELECT` (two deep this time!) and a `CASE` statement. The `CASE` keyword acts as an if statement and returns specific values in specific cases. For us, anything larger than an 8 (i.e. 9 or 10) is a promoter, otherwise, anything larger than a 6 (i.e. 7 or 8) is a passive and everything else is a detractor:

```SQL
SELECT * FROM
(SELECT CASE
    WHEN avg_week_score > 8 THEN 'promoter'
    WHEN avg_week_score > 6 THEN 'passive'
    ELSE 'detractor'
END AS nps_class, week FROM
(SELECT TO_CHAR(score.created_at, 'IYYY-IW') AS week, customer_id, AVG(score) as avg_week_score FROM score
GROUP BY week, customer_id) a) b
limit 10;
```

Which gives us the following output: a huge table with the nps_class and the week number:

```
 nps_class |  week
-----------+---------
 detractor | 2018-01
 detractor | 2018-01
 promoter  | 2018-01
 detractor | 2018-01
 promoter  | 2018-01
 detractor | 2018-01
 detractor | 2018-01
 detractor | 2018-01
 detractor | 2018-01
 promoter  | 2018-01
```

This is closer to what we need, but not very useful in its current form. We can confirm that there are still nearly a million rows by using another `COUNT`:

```SQL
SELECT count(*) FROM
(SELECT CASE
    WHEN avg_week_score > 8 THEN 'promoter'
    WHEN avg_week_score > 6 THEN 'passive'
    ELSE 'detractor'
END AS nps_class, week FROM
(SELECT TO_CHAR(score.created_at, 'IYYY-IW') AS week, customer_id, AVG(score) as avg_week_score FROM score
GROUP BY week, customer_id) a) b
limit 10;
```

```
 count
--------
 951289
(1 row)
```

Now that we've broken our customers into specific categories, we want to count them. It's useful to "pivot" this data so that we can see the count of each class of people as a separate column. In a spreadsheet program like Microsoft Excel or Google Sheets, we would think of this as a pivot table, and there are plugins for PostgreSQL to allow you to use it in a similar way. In our case, though, we can count the number of each class each week using some more `CASE` statements and the `SUM` function as follows:

```SQL
SELECT week,
SUM(CASE WHEN nps_class = 'promoter' THEN 1 ELSE 0 END) AS "promoter",
SUM(CASE WHEN nps_class = 'passive' THEN 1 ELSE 0 END) AS "passive",
SUM(CASE WHEN nps_class = 'detractor' THEN 1 ELSE 0 END) AS "detractor",
    COUNT(*) AS "total" FROM
(SELECT CASE
    WHEN avg_week_score > 8 THEN 'promoter'
    WHEN avg_week_score > 6 THEN 'passive'
    ELSE 'detractor'
END AS nps_class, week FROM
(SELECT TO_CHAR(score.created_at, 'IYYY-IW') AS week, customer_id, AVG(score) as avg_week_score FROM score
GROUP BY week, customer_id) a) b
GROUP BY week
ORDER BY week
limit 100;
```

Which results in (truncated for brevity):

```
  week   | promoter | passive | detractor | total
---------+----------+---------+-----------+-------
 2018-01 |       26 |       0 |        39 |    65
 2018-02 |       65 |       2 |        63 |   130
 2018-03 |       71 |       7 |        70 |   148
 2018-04 |       76 |       6 |        83 |   165
 2018-05 |      186 |      23 |       135 |   344
 2018-06 |      397 |      56 |       202 |   655
 2018-07 |      471 |      72 |       238 |   781
 2018-08 |      520 |      79 |       276 |   875
 2018-09 |      771 |     102 |       300 |  1173
 2018-10 |     1154 |     154 |       351 |  1659
 ...
```

Note that we also had to add another intermediate alias (`b`) to our SQL code, as we have yet another level of nested `SELECT`.

### Calculating NPS per week (10 min)

We now have all the pieces in place to calculate our NPS. To do this, we will have to use a *third* nested `SELECT` and yet another table alias `c`.

### Exercise 12: (10 min)

Given the above guidance, write the query to compute NPS per week.

**Answer.** One possible solution is given below:

```SQL
SELECT *, ROUND(((CAST(promoter AS DECIMAL) / total) - (CAST(detractor AS DECIMAL) / total)) * 100, 0) AS nps FROM
(SELECT week,
SUM(CASE WHEN nps_class = 'promoter' THEN 1 ELSE 0 END) AS "promoter",
SUM(CASE WHEN nps_class = 'passive' THEN 1 ELSE 0 END) AS "passive",
SUM(CASE WHEN nps_class = 'detractor' THEN 1 ELSE 0 END) AS "detractor",
    COUNT(*) AS "total" FROM
(SELECT CASE
    WHEN avg_week_score > 8 THEN 'promoter'
    WHEN avg_week_score > 6 THEN 'passive'
    ELSE 'detractor'
END AS nps_class, week FROM
(SELECT TO_CHAR(score.created_at, 'IYYY-IW') AS week, customer_id, AVG(score) as avg_week_score FROM score
GROUP BY week, customer_id) a) b
GROUP BY week
ORDER BY week) c
limit 100;
```



```
  week   | promoter | passive | detractor | total | nps
---------+----------+---------+-----------+-------+-----
 2018-01 |       26 |       0 |        39 |    65 | -20
 2018-02 |       65 |       2 |        63 |   130 |   2
 2018-03 |       71 |       7 |        70 |   148 |   1
 2018-04 |       76 |       6 |        83 |   165 |  -4
 2018-05 |      186 |      23 |       135 |   344 |  15
 2018-06 |      397 |      56 |       202 |   655 |  30
 2018-07 |      471 |      72 |       238 |   781 |  30
 2018-08 |      520 |      79 |       276 |   875 |  28
 2018-09 |      771 |     102 |       300 |  1173 |  40
 2018-10 |     1154 |     154 |       351 |  1659 |  48
 2018-11 |     1313 |     180 |       394 |  1887 |  49
 2018-12 |     1419 |     204 |       423 |  2046 |  49
```

That first line is not pretty, but it works! We can now see the NPS, correctly rounded, for any given week.

## Conclusions (5 min)

In the first part of this case, you learned the basics of SQL and used it to optimize the sales operations of a financial services firm. We narrowed down our set of potentially interesting customer cohorts and were able to compute summary statistics on the sales conversion rates of those cohorts, particularly versus the mean. In particular, we learned that some of our "no-brainer" hypotheses did not pan out, which illustrates the importance of always investigating the data to validate our thoughts.

In the second part of this case, you learned about the Net Promoter Score (NPS) metric. You set up a cloud database using Amazon RDS, a service that makes it easy to manage and scale your databases with little to no work from your local machine. You also learned how to write complex queries in SQL that could be run directly on the cloud database. These queries used advanced features like nested `SELECT` statements and `CASE` statements which can be combined in intricate ways to get the results you need directly from your database.

We found that there was a general increase in NPS over time; however, starting in September there was a significant downturn in average NPS score. It is likely that the product encountered some significant bugs or outages during this time and going forward we should check if anything was recorded by the startup's product team to confirm this.

## Takeaways (5 min)

SQL is a powerful tool that can help us navigate and understand data in ways that Python cannot. Sometimes, it can even serve as the first stage of an exploratory data analysis and can sometimes help us answer questions all by itself. Furthermore, SQL is the means through which we can create and persist data in databases for future, large-scale use. No data scientist's toolkit is complete without an understanding of how to interface with and store the raw data that they work with.

Additionally, cloud databases are a powerful and scalable way to analyze data if you have constraints on processor, memory or storage resources for your local hardware. You can do all sorts of things in-cloud that you could originally only do on your local machine, such as run complex SQL queries directly against a cloud database.

## Appendix 1: Troubleshooting RDS creation

If you cannot create your database using the RDS service and instead see the error below, you will need to create a new VPC instead of using the default one. 

![vpc dns error](images/vpc-rds-error.png)


To do this, scroll back up to the 'Connectivity' section, and choose 'create new VPC' from the dropdown as shown in the image below

![create new vpc](images/create-new-vpc.png)

At the bottom of the page, press "Create Database" again, and you should see a notification briefly at the top of the page that confirms a new VPC has been created, as in the image below. Take a note of the ID.

![view vpc](images/view-vpc-id.png)

You might now see another error, as follows. This is because the VPC created from the RDS console has no name.

![vpc no name error](images/vpc-no-name-error.png)


If this is the case, you need to name your VPC. From the services dropdown at the top of the page, search for "VPC" and open the VPC page in a new tab.


![view VPCs](images/services-select-vpc.png)

Find the VPC that was recently created (it will have the same ID as the one you noted above). Mouse over the 'name' field to see the pencil 'edit' option appear, click on this, and give the VPC a name.


![name VPC](images/name-vpc.png)

Now that your VPC has a name, go back to the tab where you are creating the RDS instance, and scroll back up to the connectivity section, and choose the newly created VPC (you will see the name you chose displayed) from the dropdown.

Now you can finally press "Create database" again (at the bottom of the page) and all should work.